# 329 - Prime Frog

## Problem Statement

Susan has a prime frog.<br>
Her frog is jumping around over $500$ squares numbered $1$ to $500$.
He can only jump one square to the left or to the right, with equal probability, and he cannot jump outside the range $[1;500]$.<br>(if it lands at either end, it automatically jumps to the only available square on the next move.)

When he is on a square with a prime number on it, he croaks 'P' (PRIME) with probability $2/3$ or 'N' (NOT PRIME) with probability $1/3$ just before jumping to the next square.<br>
When he is on a square with a number on it that is not a prime he croaks 'P' with probability $1/3$ or 'N' with probability $2/3$ just before jumping to the next square.

Given that the frog's starting position is random with the same probability for every square, and given that she listens to his first $15$ croaks, what is the probability that she hears the sequence PPPPNNPPPNPPNPN?

Give your answer as a fraction $p/q$ in reduced form.

## Solution

This can be solved with dynamic programming. We define the states as $f(i, k)$ where $i$ represents the current square number, $k$ represents the number of jumps remaining. $f(i, k)$ represents the probability of obtaining the last $k$ letters of the sequence from position $i$. Working backwards, when $k = 0$, the probability of obtaining the last letter of the sequence (N) is $\frac{2}{3}$ when $i$ is primes and $\frac{1}{3}$ otherwise. In the case where letter $15 - k$ is P we have

\begin{equation}
    f(i, k) = \left[\mathbb{1}_{i \in \mathcal{P}} \left(\frac{2}{3} \right) + \mathbb{1}_{i \notin \mathcal{P}} \left(\frac{1}{3} \right) \right] \left( \frac{1}{2}f(i - 1, k - 1) + \frac{1}{2}f(i + 1, k - 1) \right), \qquad 1 < i < 500
\end{equation}

where $\mathcal{P}$ is the set of prime numbers and $\mathbb{1}_{x}$ is the indicator function taking value 1 if condition $x$ is met and 0 otherwise.

The cases when $15 - k$ is N or $i$ is 0 or 500 are straightforward to deduce from the equation above.

Based on this we implement a dynamic programming solution. When processing a given $k$, we only need the probabilities from the previous $k$ value. Therefore, we keep our ```dp``` table one dimensional. Once we obtain the final vector, we simply need take the average of the 500 values to obtain the solution.

In [4]:
import sympy
import numpy as np
from fractions import Fraction

sequence = 'PPPPNNPPPNPPNPN'
primes = set(sympy.primerange(500))
dp = [1] * 500
for k in range(14, -1, -1):
    prev_dp = dp
    dp = [0] * 500
    for i in range(500):
        if i + 1 in primes and sequence[k] == 'P':
            dp[i] = Fraction(2, 3) * Fraction(1, 2) * (prev_dp[i - 1] + prev_dp[i + 1])
        elif i + 1 in primes and sequence[k] == 'N':
            dp[i] = Fraction(1, 3) * Fraction(1, 2) * (prev_dp[i - 1] + prev_dp[i + 1])
        elif sequence[k] == 'P':
            if i == 0:
                dp[i] = Fraction(1, 3) * prev_dp[i + 1]
            elif i == 499:
                dp[i] = Fraction(1, 3) * prev_dp[i - 1]
            else:
                dp[i] = Fraction(1, 3) * Fraction(1, 2) * (prev_dp[i - 1] + prev_dp[i + 1])
        elif sequence[k] == 'N':
            if i == 0:
                dp[i] = Fraction(2, 3) * prev_dp[i + 1]
            elif i == 499:
                dp[i] = Fraction(2, 3) * prev_dp[i - 1]
            else:
                dp[i] = Fraction(2, 3) * Fraction(1, 2) * (prev_dp[i - 1] + prev_dp[i + 1])

proba = np.mean(dp)

print(f'{proba.numerator}/{proba.denominator}')

199740353/29386561536000
